In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
pd.set_option('display.max_columns', None, 'max_colwidth', 250)

import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gravi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
reddit = pd.read_csv('comment_plus_parent.csv', index_col=0)
reddit.head()

,label,comment,parent_comment
0,0,NC and NH.,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well."
1,0,You do know west teams play against west teams more than east teams right?,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.
2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",They're favored to win.
3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",deadass don't kill my buzz
4,0,I could use one of those tools.,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT


In [3]:
reddit.shape

(1010714, 3)

In [4]:
y = reddit.pop('label')
X = reddit

In [5]:
X.shape

(1010714, 2)

In [6]:
y.shape

(1010714,)

In [7]:
# first split into test and remainder
X_remainder, X_test, y_remainder, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

In [8]:
# second split into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_remainder, y_remainder, test_size = 0.25, random_state=42)

In [9]:
stemmer = nltk.stem.PorterStemmer()
stop_words = stopwords.words('english')

def tokenizer(sentence):
    
    num_range = range(0,10**3)
    
    for num in num_range:
        sentence = sentence.replace(str(num), '')
    
    # remove punctuation and set to lower case
    for punctuation_mark in string.punctuation:
        sentence = sentence.replace(punctuation_mark,'').lower()
        
    
    # split sentence into words
    words_list = sentence.split(' ')
    stemmed_words = []
    
    
    # remove stopwords and any tokens that are just empty strings
    for word in words_list:
        if (not word in stop_words) and (word!=''):
            # Stem words
            stemmed_word = stemmer.stem(word)
            stemmed_words.append(stemmed_word)

    return stemmed_words

In [10]:
# Instantiating transformers to pass into column transformer
transformers = [('comment', CountVectorizer(min_df=25, tokenizer=tokenizer, max_features=750, dtype=np.int8), 'comment'),
                 ('parent', CountVectorizer(min_df=25, tokenizer=tokenizer, max_features=750, dtype=np.int8), 'parent_comment')]
                

# Creating the column transformer
column_transform = ColumnTransformer(transformers)

# Fitting and transforming train, test, val
X_train_tokens = column_transform.fit_transform(X_train)
X_val_tokens = column_transform.transform(X_val)
X_test_tokens = column_transform.transform(X_test)

C:\Users\gravi\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Potential random sampling:

In [11]:
train_vectors = pd.DataFrame(columns=column_transform.get_feature_names_out(), data=X_train_tokens.toarray())
train_vectors.shape

(568526, 1500)

In [13]:
val_vectors = pd.DataFrame(columns=column_transform.get_feature_names_out(), data=X_val_tokens.toarray())
val_vectors.shape

(189509, 1500)

In [12]:
test_vectors = pd.DataFrame(columns=column_transform.get_feature_names_out(), data=X_test_tokens.toarray())
test_vectors.shape

(252679, 1500)

In [14]:
logreg = LogisticRegression(max_iter=5000, random_state=42)

logreg.fit(train_vectors, y_train)

print(f'Train score: {logreg.score(train_vectors, y_train)}')
print(f'Val score: {logreg.score(val_vectors, y_val)}')
print(f'Val score: {logreg.score(test_vectors, y_test)}')

Train score: 0.6529270429144841
Val score: 0.6496841838646186
Val score: 0.6502756461755824
